<h3>Importing the Libraries

In [1]:
import os
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
from copy import deepcopy
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR

<h3>Manually performing OneHot Encoding on Alphabets and 'eon'

In [2]:
alphabets = ["eon", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"]
encoded_alphabets = np.zeros((len(alphabets), len(alphabets)))
for i in range(len(alphabets)):
    encoded_alphabets[i][i] = 1

<h3>Reading Input Text File

In [3]:
for files in os.listdir():
        filename, extension = os.path.splitext(files)
        if extension == '.txt':
            input_filename = files
with open(input_filename) as f:
    lines = f.readlines()

words = np.zeros((len(lines), 11), dtype='U16')

for name_index in range(len(lines)):
    name = lines[name_index].lower().strip()
    for character_index in range(len(name)):
        words[name_index][character_index] = name[character_index]
    for empty_index in range(len(name), 11):
        words[name_index][empty_index] = 'eon'
        
words_copy = deepcopy(words)

In [4]:
desired_output = deepcopy(words)
for i in range(len(words_copy)):
    temp_array = np.delete(words_copy[i], 0)
    desired_output[i] = np.append(temp_array, 'eon')

In [5]:
inputs = np.ones((2000, 11, 27), dtype = 'double')
for name_index in range(len(words)):
    for char_index in range(len(words[name_index])):
        character = words[name_index][char_index]
        encoded_index = alphabets.index(character)
        inputs[name_index][char_index] = encoded_alphabets[encoded_index]

In [6]:
outputs = np.ones((2000, 11, 27), dtype = 'double')
for name_index in range(len(words)):
    for char_index in range(len(words[name_index])):
        character = words[name_index][char_index]
        encoded_index = alphabets.index(character)
        outputs[name_index][char_index] = encoded_alphabets[encoded_index]

<h3>Creating Tensors for Inputs and Desired Outputs

In [7]:
inputs = torch.tensor(inputs).float()
outputs = torch.tensor(outputs).float()

<h3>Declaring Custom Dataset Class

In [8]:
class CustomDataset(Dataset):
    def __init__(self):
        pass
        
    def __len__(self):
        return len(inputs)
    
    def __getitem__(self, index):
        return inputs[index]

<h3>Initializing DataLoader

In [9]:
train_loader = DataLoader(dataset = inputs, batch_size = 1, shuffle = False)

<h3>Declaring Neural Network Layers for LSTM Operation

In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hidden_1 = nn.LSTM(27, 27)
        self.fc1 = nn.Linear(27, 27)
        self.fc2 = nn.Linear(27, 27)
        
    def forward(self, output):
        output, hidden = self.hidden_1(output)
        output = self.fc1(output)
        output = self.fc2(output)
        return output, hidden

<h3>Loading Trained Model

In [11]:
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = (1e-4))

model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

<h3>Function to Return OneHot Encoded Output

In [12]:
def get_encodedvalue(input_character):
    encodedvalue = []
    for entry in input_character:
        char_index = alphabets.index(entry)
        encodedvalue.append(encoded_alphabets[char_index])
    return encodedvalue

<h3>Function to Return Characters from OneHot Encoded values

In [13]:
def get_charactervalue(obtained_output):
    charactervalue = []
    value, indices = torch.topk(obtained_output, 4, dim = 2)
    x, y, z = indices.size()
    random_number = np.random.random()
    if random_number > 0.9:
        rand_int = 0
    elif random_number > 0.5 and random_number < 0.9:
        rand_int = 1
    elif random_number > 0.3 and random_number < 0.5:
        rand_int = 2
    else:
        rand_int = 3      
    if (alphabets[indices[0][-1][rand_int]]) == "eon":
        flag = False
        return charactervalue, flag
    charactervalue.append(alphabets[indices[0][-1][rand_int]])
    flag = True
    return charactervalue, flag

In [14]:
output_tracker = []
counter = 0
while counter < 20:
    flag = True
    obtained_char = []
    input_character = []
    input_character.append("a")
    while flag == True:
        encoded_input = get_encodedvalue(input_character)
        encoded_input_tensor = torch.tensor(encoded_input).float()
        output, hidden = model(encoded_input_tensor.unsqueeze(0))
        obtained_char, flag = get_charactervalue(output)
        input_character = input_character + obtained_char
        if input_character[-1] == "eon":
            flag = False
        if len(input_character) > 11:
             flag = False
    for index, character in enumerate(input_character):
        if index == (len(input_character) - 1):
            if character == input_character[index - 1]:
                continue
        elif index == 0:
            if character == input_character[index + 1]:
                continue
        elif character == input_character[index + 1]:
            continue
        elif character == input_character[index - 1]:
            continue
    if (''.join(input_character)) in output_tracker:
        continue
    if (len(''.join(input_character)) < 3):
        continue
    print(''.join(input_character))
    output_tracker.append(str(''.join(input_character)))
    counter +=1

anlelanlai
anlaai
aioa
anlea
anai
anananlnla
anlai
aioii
anla
aiana
aioe
aaa
aii
anln
aioaaia
aioelelelann
anlaia
aiaa
anlaioa
aan


<ipython-input-14-d657cf9558c1>:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)
  encoded_input_tensor = torch.tensor(encoded_input).float()


In [15]:
output_tracker = []
counter = 0
while counter < 20:
    flag = True
    obtained_char = []
    input_character = []
    input_character.append("x")
    while flag == True:
        encoded_input = get_encodedvalue(input_character)
        encoded_input_tensor = torch.tensor(encoded_input).float()
        output, hidden = model(encoded_input_tensor.unsqueeze(0))
        obtained_char, flag = get_charactervalue(output)
        input_character = input_character + obtained_char
        if input_character[-1] == "eon":
            flag = False
        if len(input_character) > 11:
             flag = False
    if (''.join(input_character)) in output_tracker:
        continue
    if (len(''.join(input_character)) < 3):
        continue
    print(''.join(input_character))
    output_tracker.append(str(''.join(input_character)))
    counter +=1

xcyn
xdd
xxc
xdt
xsroa
xxcye
xcn
xxxccnlna
xcnla
xdia
xxd
xxxc
xxcynln
xsioellnana
xcyeln
xxdi
xsriaa
xxxd
xdtt
xxdiioan
